# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
# check gpu type
!nvidia-smi

Tue Aug 13 11:34:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.18                 Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T600 Laptop GPU       WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8              N/A / ERR! |    735MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm.notebook import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [3]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [4]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [5]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 8),
            nn.LeakyReLU(),          
            nn.Linear(8, 1)
        )
        # 使用BatchNorm和LeakyReLU
        # self.layers = nn.Sequential(
        #     nn.Linear(input_dim, 16),
        #     nn.BatchNorm1d(16),
        #     nn.LeakyReLU(),
        #     nn.Linear(16, 16),
        #     nn.BatchNorm1d(16),
        #     nn.LeakyReLU(),
        #     nn.Linear(16, 1)
        # )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

In [6]:
class My_Model_2(nn.Module):
    def __init__(self, input_dim):
        super(My_Model_2, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 16),
            nn.LeakyReLU(),          
            nn.Linear(16, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

In [7]:
class My_Model_3(nn.Module):
    def __init__(self, input_dim):
        super(My_Model_3, self).__init__()
        # self.layers = nn.Sequential(
        #     nn.Linear(input_dim, 16),
        #     nn.LeakyReLU(),
        #     nn.Linear(16, 8),
        #     nn.LeakyReLU(),          
        #     nn.Linear(8, 8),
        #     nn.LeakyReLU(),          
        #     nn.Linear(8, 1)
        # )
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            #nn.Dropout(p=0.3),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

In [8]:
        # self.layers = nn.Sequential(
        #     nn.Linear(input_dim, 32),
        #     #nn.BatchNorm1d(32),
        #     nn.ReLU(),
        #     nn.Linear(32, 16),
        #     nn.ReLU(),
        #     #nn.Dropout(p=0.3),
        #     nn.Linear(16, 8),
        #     nn.ReLU(),
        #     nn.Linear(8, 1)
        # )

# Feature Selection
Choose features you deem useful by modifying the function below.

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression

def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # feat_idx = list(range(35, 88)) # TODO: Select suitable feature columns.
        k = 24
        selector = SelectKBest(score_func=f_regression, k=k)
        result = selector.fit(train_data[:, :-1], train_data[:,-1])
        idx = np.argsort(result.scores_)[::-1]
        feat_idx = list(np.sort(idx[:k]))
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [10]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=1e-5)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.99, patience=50) 
    # writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in tqdm(range(n_epochs)):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        # train_pbar = tqdm(train_loader, position=0, leave=True)

        # for x, y in train_pbar:
        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            # train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            # train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        # writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        if epoch+1 % 100 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        # # 更新学习率
        # scheduler.step(mean_valid_loss)
        # writer.add_scalar('Learning Rate', optimizer.param_groups[0]['lr'], step)
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            # print('Saving model with loss {:.4f}...at epoch {}'.format(best_loss, epoch+1))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return best_loss
    
    return best_loss

In [11]:
def trainer_3(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    #criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    #optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7, weight_decay=1e-5) 
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.8, 0.999), weight_decay=1e-5, amsgrad=True)
    
    # writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in tqdm(range(n_epochs)):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        # train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            # train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            # train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        # writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        # print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            # print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return best_loss
        
    return best_loss

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 10000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 1e-4,              
    'early_stop': 2000,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/boss.ckpt'  # Your model will be saved here.
}


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [13]:
same_seed(config['seed'])
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Normalize features.

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])


# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2408, 89) 
valid_data size: (601, 89) 
test_data size: (997, 88)
number of features: 24


# Start training!

In [14]:
# model_2 = My_Model_2(x_train.shape[1]).to(device)
# config['save_path'] = './models/boss_2.ckpt'
# best_loss_2 = trainer(train_loader, valid_loader, model_2, config, device)
# print(f'Best validation loss of Model_2: {best_loss_2:.4f}')

In [15]:
# model_3 = My_Model_3(x_train.shape[1]).to(device)
# config['save_path'] = './models/boss_3.ckpt'
# best_loss_3 = trainer_3(train_loader, valid_loader, model_3, config, device)
# print(f'Best validation loss of Model_3: {best_loss_3:.4f}')

In [16]:
# K折交叉验证训练模型
K = 5 # 因为valid_ratio=0.2，所以K=5
same_seed(config['seed'])
training_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
num_valid_samples = len(training_data) // K
np.random.shuffle(training_data)
valid_scores = []
for fold in range(K):
        print(f'Fold {fold + 1}')
        config['save_path'] = f'./models/boss_folds/boss_{fold}fold.ckpt'
        valid_data = training_data[num_valid_samples * fold:
                                num_valid_samples * (fold + 1)]
        train_data = np.concatenate((
            training_data[:num_valid_samples * fold],
            training_data[num_valid_samples * (fold + 1):]))
        # Select features
        x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

        # Print out the number of features.
        print(f'number of features: {x_train.shape[1]}')

        train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                                    COVID19Dataset(x_valid, y_valid), \
                                                    COVID19Dataset(x_test)

        # Pytorch data loader loads pytorch dataset into batches.
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

        model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
        score = trainer(train_loader, valid_loader, model, config, device)
        valid_scores.append(score)
        print(f'Fold {fold + 1} valid_loss: {score}')

print(f'Best validation loss of Model: {np.min(valid_scores):.4f}')

Fold 1
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 1 valid_loss: 0.7020852466424307
Fold 2
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 2 valid_loss: 0.79047363003095
Fold 3
number of features: 24


...


Model is not improving, so we halt the training session.
Fold 3 valid_loss: 0.8514037529627482
Fold 4
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 4 valid_loss: 0.8579290310541788
Fold 5
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 5 valid_loss: 0.7549566924571991
Best validation loss of Model: 0.7021


In [17]:
K = 5 # 因为valid_ratio=0.2，所以K=5
same_seed(config['seed'])
training_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
num_valid_samples = len(training_data) // K
np.random.shuffle(training_data)
valid_scores_2 = []
for fold in range(K):
        print(f'Fold {fold + 1}')
        config['save_path'] = f'./models/boss_folds/boss2_{fold}fold.ckpt'
        valid_data = training_data[num_valid_samples * fold:
                                num_valid_samples * (fold + 1)]
        train_data = np.concatenate((
            training_data[:num_valid_samples * fold],
            training_data[num_valid_samples * (fold + 1):]))
        # Select features
        x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

        # Print out the number of features.
        print(f'number of features: {x_train.shape[1]}')

        train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                                    COVID19Dataset(x_valid, y_valid), \
                                                    COVID19Dataset(x_test)

        # Pytorch data loader loads pytorch dataset into batches.
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

        model = My_Model_2(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
        score = trainer(train_loader, valid_loader, model, config, device)
        valid_scores_2.append(score)
        print(f'Fold {fold + 1} valid_loss: {score}')

print(f'Best validation loss of Model_2: {np.min(valid_scores_2):.4f}')

Fold 1
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 1 valid_loss: 0.6801783243815104
Fold 2
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 2 valid_loss: 0.8180224895477295
Fold 3
number of features: 24


...


Model is not improving, so we halt the training session.
Fold 3 valid_loss: 0.8221958180268606
Fold 4
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 4 valid_loss: 0.8600337902704874
Fold 5
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]


Model is not improving, so we halt the training session.
Fold 5 valid_loss: 0.7526910901069641
Best validation loss of Model_2: 0.6802


In [18]:
K = 5 # 因为valid_ratio=0.2，所以K=5
same_seed(config['seed'])
training_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
num_valid_samples = len(training_data) // K
np.random.shuffle(training_data)
valid_scores_3 = []
for fold in range(K):
        print(f'Fold {fold + 1}')
        config['save_path'] = f'./models/boss_folds/boss3_{fold}fold.ckpt'
        valid_data = training_data[num_valid_samples * fold:
                                num_valid_samples * (fold + 1)]
        train_data = np.concatenate((
            training_data[:num_valid_samples * fold],
            training_data[num_valid_samples * (fold + 1):]))
        # Select features
        x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

        # Print out the number of features.
        print(f'number of features: {x_train.shape[1]}')

        train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                                    COVID19Dataset(x_valid, y_valid), \
                                                    COVID19Dataset(x_test)

        # Pytorch data loader loads pytorch dataset into batches.
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

        model = My_Model_3(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
        score = trainer_3(train_loader, valid_loader, model, config, device)
        valid_scores_3.append(score)
        print(f'Fold {fold + 1} valid_loss: {score}')

print(f'Best validation loss of Model_3: {np.min(valid_scores_3):.4f}')

Fold 1
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 1 valid_loss: 0.6957943737506866
Fold 2
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 2 valid_loss: 0.8669416705767313
Fold 3
number of features: 24


...

Fold 3 valid_loss: 0.868886411190033
Fold 4
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 4 valid_loss: 0.8762156963348389
Fold 5
number of features: 24


  0%|          | 0/10000 [00:00<?, ?it/s]

Fold 5 valid_loss: 0.7980795303980509
Best validation loss of Model_3: 0.6958


In [19]:
# # 使用随机森林进行预测
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
# train_data, valid_data = train_valid_split(train_data, 0.8, config['seed'])
# x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# model_rf = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=config['seed'])
# model_rf.fit(x_train, y_train)
# y_pred_valid = model_rf.predict(x_valid)
# valid_score = mean_squared_error(y_valid, y_pred_valid)
# print(f'RandomForest valid_score: {valid_score}')

# # 支持向量机
# from sklearn.svm import SVR
# model_svr = SVR()
# model_svr.fit(x_train, y_train)
# y_pred_valid = model_svr.predict(x_valid)
# valid_score = mean_squared_error(y_valid, y_pred_valid)
# print(f'SVR valid_score: {valid_score}')

# # 梯度提升
# from sklearn.ensemble import GradientBoostingRegressor
# model_gbr = GradientBoostingRegressor(n_estimators=10000, random_state=config['seed'])
# model_gbr.fit(x_train, y_train)
# y_pred_valid = model_gbr.predict(x_valid)
# valid_score = mean_squared_error(y_valid, y_pred_valid)
# print(f'Gradient Boosting valid_score: {valid_score}')              

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [20]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
idx = np.argmin(valid_scores)
load_path = f'./models/boss_folds/boss_{idx}fold.ckpt'
model.load_state_dict(torch.load(load_path))
preds = predict(test_loader, model, device) 

model_2 = My_Model_2(input_dim=x_train.shape[1]).to(device)
idx = np.argmin(valid_scores_2)
load_path = f'./models/boss_folds/boss2_{idx}fold.ckpt'
model_2.load_state_dict(torch.load(load_path))
preds_2 = predict(test_loader, model_2, device)

model_3 = My_Model_3(input_dim=x_train.shape[1]).to(device)
idx = np.argmin(valid_scores_3)
load_path = f'./models/boss_folds/boss3_{idx}fold.ckpt'
model_3.load_state_dict(torch.load(load_path))
preds_3 = predict(test_loader, model_3, device)

# 使用集成预测
# preds_rf = model_rf.predict(x_test)
# preds_svm = model_svr.predict(x_test)
# preds_gbr = model_gbr.predict(x_test)
preds = (preds + preds_2 + preds_3) / 3
save_pred(preds, 'boss.csv')

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)